In [2]:
import os
import zipfile
import copy


class Person(object):
    def __init__(self, name=None, number=None):
        self.name = name
        self.num = number


class Reader(object):
    def __init__(self, path):
        self.mode = path

    def read(self):
        raise Exception("子类没有read方法")


class CsvReader(Reader):
        # csv似乎不能用utf-8
    def __init__(self, path, mode):
        self.path = path
        self.mode = mode

    def read(self):
        data = []
        with open(self.path, self.mode, encoding='gbk') as f:
            for line in f.readlines():
                line = line.rstrip()
                persondata = line.split()
                name = persondata[0]
                num = persondata[1]
                data.append(Person(name, num))
            return data


class TxtReader(Reader):
    def __init__(self, path, mode):
        self.path = path
        self.mode = mode

    def read(self):
        data = []
        with open(self.path, self.mode, encoding='UTF-8') as f:
            for line in f.readlines():
                line = line.rstrip()
                persondata = line.split()
                name = persondata[0]
                num = persondata[1]
                data.append(Person(name, num))
            return data


class ZipReader(TxtReader, CsvReader):
    def __init__(self, path, mode):
        self.path = path
        self.mode = mode

    def read(self):
        z_file = zipfile.ZipFile(self.path, self.mode)
        flist = z_file.namelist()
        filepath = z_file.extract(flist[0])
        filename, filetype = self.read_filename(filepath)
        data = self.readfile(filepath, filetype)
        return data

    @staticmethod
    def read_filename(path):
        file = os.path.splitext(path)
        filename, filetype = file
        return filename, filetype

    # 使用dict代替 if语句
    def readfile(self, filepath, filetype):
        readway = {'.txt': TxtReader, '.csv': CsvReader}
        for i in readway.keys():
            if i == filetype:
                reader = readway[filetype]
                obj = reader(filepath, self.mode)
                data = obj.read()
                return data
        else:
            raise Exception("不能读取该文件类型")


# 统一接口
def read_data(obj):
    return obj.read()


class Ring(object):
    MAX_SIZE = 1000

    def __init__(self, reader=None):
        self.step = 1
        self.start = 0
        self._ring = []
        self._temp = []
        self._current_id = 0
        if reader:
            for each in reader:
                self._ring.append(each)

    def is_empty(self):
        return len(self._temp) == 0

    def pop(self, index):
        self._ring.pop(index)

    def quer_list(self):
        return self._ring

    def reset(self, step, location):
        self.step = step
        self.start = location-1
        self._current_id = self.start
        self._temp = copy.deepcopy(self._ring)
        return

    def next(self):
        if self.is_empty() is True:
            raise Exception('环为空')
        size = len(self._temp)
        id = (self._current_id+(self.step-1)) % size
        outelem = self._temp.pop(id)
        self._current_id += (self.step - 1)
        return outelem

    def popelem(self):
        step = self.step
        temp = self._temp
        location = self.start % len(temp)
        if self.is_empty() is True:
            raise Exception('环为空')

        for each in range(0, len(temp)):
            location = (location + (step-1)) % len(temp)
            outelem = temp.pop(location)
            yield outelem

    def append(self, obj):
        if len(self._ring) > Ring.MAX_SIZE:
            raise Exception("out of range")
        self._ring.append(obj)


try:
    steps = int(input("数几个数"))
    if steps <= 0:
        raise Exception('数的数必须大于1')
    loc = int(input("开始位置"))
    if loc <= 0:
        raise Exception('位置必须大于1')

    readdata = ZipReader('peopledata.zip', 'r')
    peopledata  = read_data(readdata)
    jospeh = Ring(peopledata)
    jospeh.reset(steps, loc)

except ValueError:
    print('输入错误，输入的不是数字')
else:
    popseq = jospeh.popelem()
    for popperson in popseq:
        print(popperson.name, popperson.num)







金迎夜 5
凤千谷 8
冯巧 1
吕天 4
杨岚 9
袁春绿 3
陶晓 10
尤文白 7
昌瑶灵 2
潘海曼 6
